In [ ]:
!pip install pypulseq==1.3.1.post1 &> /dev/null
!pip install MRzeroCore &> /dev/null
!wget https://github.com/MRsources/MRzero-Core/raw/main/documentation/playground_mr0/numerical_brain_cropped.mat &> /dev/null

(DREAM_STE_seq)=
# STE - DREAM

This is a pulseq implementation of the single slice DREAM sequence [1] using the STE first timing scheme and thus enabling the mapping of B1, B0 and transceive chain phase.

---

[1] Nehrke, K. and Börnert, P. (2012), DREAM—a novel approach for robust, ultrafast, multislice B1 mapping. Magn Reson Med, 68: 1517-1526. https://doi.org/10.1002/mrm.24158

In [ ]:
#@title generate
# %% S0. SETUP env
import MRzeroCore as mr0
import pypulseq as pp
import numpy as np
import torch
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

## import for masking
import matplotlib.image

## imports for image reconstruction
from skimage.restoration import unwrap_phase

## import for fitting
from scipy.stats import linregress

experiment_id = 'DREAM_STE'

# %% S1. SETUP sys
'''
# choose the scanner limits
system = pp.Opts(max_grad=28,grad_unit='mT/m',max_slew=150,slew_unit='T/m/s',
                 rf_ringdown_time=20e-6,rf_dead_time=100e-6,adc_dead_time=20e-6,grad_raster_time=50*10e-6)
'''

system = pp.Opts(
    max_grad=80,
    grad_unit='mT/m',
    max_slew=200,
    slew_unit='T/m/s',
    rf_ringdown_time=30e-6,
    rf_dead_time=100e-6,
    adc_dead_time=20e-6,
    grad_raster_time=10e-6
)

# %% S2. DEFINE the sequence
seq = pp.Sequence()

# Define FOV and resolution
fov = 200e-3
slice_thickness=8e-3
sz = (64, 64)  # spin system size / resolution
Nread = 64  # frequency encoding steps/samples
Nphase = 64  # phase encoding steps/samples
zoom = 1
ncoils = 20 #16 for: links, Mitte, Halbrechts, Rechts, Mitte_v1, DrehungHalblinks, DrehungLinks, DrehungHalbRechts, DrehungRechts  #20 for: Halblinks
t0 = 5e-4 # smallest time interval [s] used in all events
#t1 = t0#*5 # t0 has to be converted for the extended gradients
gx_read_amp = (2*Nread*zoom)/(fov*4*t0) # amplitude for G_m (gx_read) part of gx_ext
gx_pre_amp = -7*gx_read_amp/2 # amplitude for G_m1 (gx_pre) part of gx_ext
# -> maybe change the timing system, so that t0 isn't used for everything and events can be timed better individually


# Define rf events
# STEAM rf pulses:
rf1 = pp.make_block_pulse(flip_angle=55 * np.pi / 180, duration=t0, slice_thickness=slice_thickness, system=system)
rf2 = pp.make_block_pulse(flip_angle=55 * np.pi / 180, phase_offset=180*np.pi/180, duration=t0, slice_thickness=slice_thickness, system=system)

rf1.delay=0
rf2.delay=0

# FLASH readout pulse and slice selction gradients:
rf3, gz3, gzr3 = pp.make_sinc_pulse(
    flip_angle=15 * np.pi/180,
    duration=2*t0,
    slice_thickness=slice_thickness,
    apodization=0.5,
    time_bw_product=4,
    system=system,
    return_gz=True
)


# Define other gradients and ADC events
gx_ext = pp.make_extended_trapezoid(channel='x', amplitudes=np.array([0,gx_pre_amp,0,gx_read_amp,gx_read_amp,10*gx_read_amp,0]), times=np.array([0,1*t0,2*t0,3*t0,7*t0,8*t0,9*t0]), system=system)
gx_m2 = pp.make_trapezoid(channel='x', area=-gx_read_amp*2*t0, duration=11*t0+2*gz3.fall_time-system.rf_ringdown_time, system=system)
gx_spoil = pp.make_trapezoid(channel='x', area=6*gx_read_amp*t0, duration=1e-3, system=system)
adc = pp.make_adc(num_samples=Nread*2, duration=4*t0, delay=3*t0, phase_offset=0*np.pi/180,system=system)

#dummies
dummies = 3

#rf spoiling
rf_phase = 0
rf_inc = 0
rf_spoiling_inc=84

#centric reordering
phase_enc__gradmoms = torch.arange(0,Nphase,1)-Nphase//2

permvec=np.zeros((Nphase,),dtype=int)
permvec[0]=0
for i in range(1,int(Nphase//2+1)):
    permvec[i*2-1]=-i
    if i <Nphase/2:
        permvec[i*2]=i
permvec+=Nphase//2
phase_enc__gradmoms=phase_enc__gradmoms[permvec]


# ======
# CONSTRUCT SEQUENCE
# ======

'''
# MP:
rf_prep, _= make_block_pulse(flip_angle=180 * math.pi / 180, duration=1e-3, system=system)
#FLAIR
seq.add_block(rf_prep)
seq.add_block(make_delay(2.7))
seq.add_block(gx_spoil)
'''
'''
# DIR
seq.add_block(rf_prep)
seq.add_block(make_delay(0.45))
seq.add_block(gx_spoil)
'''

#STEAM block
seq.add_block(rf1)
seq.add_block(gx_m2)
seq.add_block(rf2)
seq.add_block(gx_spoil)

#dummy block
for i in range(0, dummies):
    #rf spoiling
    rf3.phase_offset = rf_phase / 180 * np.pi   # set current rf phase
    adc.phase_offset = rf_phase / 180 * np.pi  # follow with ADC
    rf_inc = divmod(rf_inc + rf_spoiling_inc, 360.0)[1]   # increase increment
    rf_phase = divmod(rf_phase + rf_inc, 360.0)[1]        # increment additional pahse
    #dummies
    seq.add_block(rf3,gz3)
    seq.add_block(gx_ext,gzr3)

#readout block
rf3.phase_offset = rf_phase / 180 * np.pi   # set current rf phase
adc.phase_offset = rf_phase / 180 * np.pi  # follow with ADC
rf_inc = divmod(rf_inc + rf_spoiling_inc, 360.0)[1]   # increase increment
rf_phase = divmod(rf_phase + rf_inc, 360.0)[1]        # increment additional pahse

seq.add_block(rf3,gz3)
seq.add_block(gx_ext,gzr3,adc)

for ii in range(1, Nphase):
    #rf spoiling
    rf3.phase_offset = rf_phase / 180.0 * np.pi   # set current rf phase
    adc.phase_offset = rf_phase / 180.0 * np.pi  # follow with ADC
    rf_inc = divmod(rf_inc + rf_spoiling_inc, 360.0)[1]   # increase increment
    rf_phase = divmod(rf_phase + rf_inc, 360.0)[1]        # increment additional pahse

    gy_amp = (2*phase_enc__gradmoms[ii]*zoom)/(fov*2*t0) # set amplitude for y-gradients
    gy_ext = pp.make_extended_trapezoid(channel='y', amplitudes=np.array([0,gy_amp,0,0,-gy_amp,0]), times=np.array([0,1*t0,2*t0,7*t0,8*t0,9*t0]), system=system)
    seq.add_block(rf3,gz3)
    seq.add_block(gx_ext,gy_ext,gzr3,adc)

#seq.add_block(pp.make_delay(4)) # delay to ensure total relaxation of magnetization when measuring multiple sequences consecutively



# %% S3. CHECK, PLOT and WRITE the sequence  as .seq
ok, error_report = seq.check_timing()  # Check whether the timing of the sequence is correct
if ok:
    print('Timing check passed successfully')
else:
    print('Timing check failed. Error listing follows:')
    [print(e) for e in error_report]

# PLOT sequence
sp_adc, t_adc = mr0.util.pulseq_plot(seq, clear=False, figid=(11,12))
#
if 0:
    sp_adc,t_adc =seq.plot(clear=True)


# Prepare the sequence output for the scanner
seq.set_definition('FOV', [fov, fov, slice_thickness])
seq.set_definition('Name', 'gre')
seq.write(experiment_id +'.seq')



# %% S4: SETUP SPIN SYSTEM/object on which we can run the MR sequence external.seq from above
sz = [64, 64]

if 1:
    # (i) load a phantom object from file
    obj_p = mr0.VoxelGridPhantom.load_mat('numerical_brain_cropped.mat')
    obj_p = obj_p.interpolate(sz[0], sz[1], 1)
    # Manipulate loaded data
    obj_p.T2dash[:] = 30e-3
    obj_p.D *= 0
    obj_p.B0 *= 1    # alter the B0 inhomogeneity
    # Store PD for comparison
    PD = obj_p.PD
    B0 = obj_p.B0
    B1 = obj_p.B1
else:
    # or (ii) set phantom  manually to a pixel phantom. Coordinate system is [-0.5, 0.5]^3
    obj_p = mr0.CustomVoxelPhantom(
        pos=[[-0.4, -0.4, 0], [-0.4, -0.2, 0], [-0.3, -0.2, 0], [-0.2, -0.2, 0], [-0.1, -0.2, 0]],
        PD=[1.0, 1.0, 0.5, 0.5, 0.5],
        T1=1.0,
        T2=0.1,
        T2dash=0.1,
        D=0.0,
        B0=0,
        voxel_size=0.1,
        voxel_shape="box"
    )
    # Store PD for comparison
    PD = obj_p.generate_PD_map()
    B0 = torch.zeros_like(PD)

obj_p.plot()
# Convert Phantom into simulation data
obj_p = obj_p.build()

# change size and orientation of phantom data for later comparison
PD.resize_(64,64)
B0.resize_(64,64)
B1.resize_(64,64)

PD=np.flip(np.rot90(PD,3),1)
B0=np.flip(np.rot90(B0,3),1)
B1=np.flip(np.rot90(B1,3),1)


In [ ]:
#@title simulate
# %% S5:. SIMULATE  the external.seq file and add acquired signal to ADC plot
seq0 = mr0.Sequence.import_file(experiment_id +'.seq')
# FIX: skip simulating z-gradients that the slice selection produces
for rep in seq0:
    rep.gradm[:, 2] = 0
seq0.plot_kspace_trajectory()
graph = mr0.compute_graph(seq0, obj_p, 200, 1e-3)
signal = mr0.execute_graph(graph, seq0, obj_p, print_progress=False)

plt.close(11);plt.close(12)
sp_adc, t_adc = mr0.util.pulseq_plot(seq, clear=False, signal=signal.numpy())


# PLOT sequence with signal in the ADC subplot
#plt.close(11);plt.close(12)
#sp_adc, t_adc = mr0.util.pulseq_plot(seq, clear=False, signal=signal.numpy()) # doesn't work for invivo
#sp_adc.plot(t_adc,np.real(signal.numpy()),t_adc,np.imag(signal.numpy()))
#sp_adc.plot(t_adc,np.abs(signal.numpy()))
'''
# additional noise as simulation is perfect
z = 1e-5*np.random.randn(signal.shape[0], 2).view(np.complex128)
signal+=z
'''



# %% S6: MR IMAGE RECON of signal ::: #####################################

fig=plt.figure(); # fig.clf()
plt.subplot(411); plt.title('ADC signal')

plt.plot(torch.real(signal),label='real')
plt.plot(torch.imag(signal),label='imag')

major_ticks = np.arange(0, 2*Nphase*Nread, Nread*2) # this adds ticks at the correct position szread
ax=plt.gca(); ax.set_xticks(major_ticks); ax.grid()

spectrum=torch.reshape((signal),(Nphase,Nread*2)).clone().transpose(1,0)

# centric reordering
kspace_adc1=spectrum[0:Nread,:]
kspace_adc2=spectrum[Nread:,:]
ipermvec=np.arange(len(permvec))[np.argsort(permvec)]
kspace1=kspace_adc1[:,ipermvec]
kspace2=kspace_adc2[:,ipermvec]

#recon of first kspace
space1 = torch.zeros_like(kspace1)
# fftshift
kspace1_1=torch.fft.fftshift(kspace1,0); kspace1_1=torch.fft.fftshift(kspace1_1,1)
#FFT
space1 = torch.fft.fft2(kspace1_1,dim=(0,1))
# fftshift
space1=torch.fft.ifftshift(space1,0); space1=torch.fft.ifftshift(space1,1)

img_STE = space1

#recon of second kspace
space2 = torch.zeros_like(kspace2)
# fftshift
kspace2_1=torch.fft.fftshift(kspace2,0); kspace2_1=torch.fft.fftshift(kspace2_1,1)
#FFT
space2 = torch.fft.fft2(kspace2_1,dim=(0,1))
# fftshift
space2=torch.fft.ifftshift(space2,0); space2=torch.fft.ifftshift(space2,1)

img_FID = space2

img_STE = np.flip(np.rot90(img_STE,3),1)

img_FID = np.flip(np.rot90(img_FID,3),1)

plt.subplot(345); plt.title('k-space_STE')
plt.imshow(np.abs(kspace1))
plt.subplot(349); plt.title('k-space_r_STE')
plt.imshow(np.log(np.abs(kspace1)))

plt.subplot(346); plt.title('k-space_FID')
plt.imshow(np.abs(kspace2))
plt.subplot(3,4,10); plt.title('k-space_r_FID')
plt.imshow(np.log(np.abs(kspace2)))

plt.subplot(347); plt.title('FFT-magnitude_STE', fontsize=15)
plt.imshow(np.abs(img_STE),vmin=0,vmax=0.083,origin='lower'); plt.axis('off'); plt.colorbar()
plt.subplot(3,4,11); plt.title('FFT-magnitude_FID', fontsize=15)
plt.imshow(np.abs(img_FID),vmin=0,vmax=0.083,origin='lower'); plt.axis('off'); plt.colorbar()

plt.subplot(348); plt.title('FFT-phase_STE', fontsize=15)
plt.imshow(np.angle(img_STE),vmin=-np.pi,vmax=np.pi,origin='lower'); plt.axis('off'); plt.colorbar()
plt.subplot(3,4,12); plt.title('FFT-phase_FID', fontsize=15)
plt.imshow(np.angle(img_FID),vmin=-np.pi,vmax=np.pi,origin='lower'); plt.axis('off'); plt.colorbar()


In [ ]:
#@title masking
# %% S7: MASKING

# choose wether or not to use a mask
masking = 1
if masking:
    # function for threshold masking
    def mask_im(input_array, threshold, mask_values):
        mask = np.ones_like(input_array)
        mask[input_array<threshold] = mask_values
        return mask

    mask_zero = mask_im(B1, 0.7, 0)
    mask_NaN = mask_im(B1, 0.7, np.NaN)

else:
    mask_zero = np.ones_like(B1)
    mask_NaN = np.ones_like(B1)

# plot mask with zeros and NaNs (zero mask is used for further calculations, NaN mask is used when showing the images)
fig=plt.figure();
plt.subplot(121); plt.title('mask_zeros', fontsize=15)
plt.imshow(mask_zero, vmin=0, vmax=1, origin='lower'); plt.colorbar()

plt.subplot(122); plt.title('mask_NaN', fontsize=15)
plt.imshow(mask_NaN, vmin=0, vmax=1, origin='lower'); plt.colorbar()


In [ ]:
#@title B1, B0, txrx
# %% S8: STEAM flip angle / B1 map
fig=plt.figure();
plt.subplot(); plt.title('B1 map [a.u.]', fontsize=15)
B1_angle = np.arctan(np.sqrt(2*(np.abs(img_STE))/(np.abs(img_FID))))*(180/np.pi)/55
plt.imshow(np.abs(B1_angle)*mask_NaN, vmin=0, vmax=1.2, origin='lower'); plt.axis('off'); plt.colorbar() # 0 to 1.2 when comparing to WASABI  # 0 to 1.09 for simulation

# %% S9: B0 maps
fig=plt.figure();
plt.subplot(); plt.title('B0 phase map [rad]', fontsize=15)
B0_phase = np.angle(img_FID*np.conjugate(img_STE))
plt.imshow(B0_phase*mask_NaN,vmin=-np.pi,vmax=np.pi, origin='lower'); plt.colorbar()

# unwrap B0 map:
fig=plt.figure();
plt.subplot(); plt.title('B0 phase map unwrapped [rad]', fontsize=15)
B0_unwrap = unwrap_phase(B0_phase*mask_zero)
plt.imshow(B0_unwrap*mask_NaN,vmin=-np.pi,vmax=np.pi, origin='lower'); plt.colorbar()

# convert to Hz
fig=plt.figure();
plt.subplot(); plt.title('B0 map [Hz]', fontsize=15)
TE = 7*t0+gz3.fall_time
#B0_freq = B0_unwrap/(2*np.pi)*(1/(2*TE))
B0_freq = B0_phase/(2*np.pi)*(1/(2*TE))
plt.imshow(B0_freq*mask_NaN, vmin=-45, vmax=45, origin='lower'); plt.axis('off'); plt.colorbar()

# %% S10: transceive chain maps
fig=plt.figure();
plt.subplot(); plt.title('$\Phi_{txrx}$ [rad]', fontsize=15)
txrx_phase = np.angle(img_FID*img_STE)
plt.imshow(txrx_phase*mask_NaN,vmin=-np.pi,vmax=np.pi, origin='lower'); plt.axis('off'); plt.colorbar()
'''
fig=plt.figure();
plt.subplot(); plt.title('$\Phi_{txrx}$ unwrapped [rad]', fontsize=15)
txrx_unwrap = unwrap_phase(txrx_phase*mask_zero)
plt.imshow(txrx_unwrap*mask_NaN,vmin=0,vmax=3*np.pi, origin='lower'); plt.colorbar()
#if unwrap starts in the wrong area and makes everything negative:
#plt.imshow((txrx_unwrap-np.min(txrx_unwrap))*mask_NaN,vmin=0,vmax=3*np.pi, origin='lower'); plt.axis('off'); plt.colorbar()
'''

In [ ]:
#@title compare DREAM and simulation phantom
# %% S11: compare DREAM simulation to simulation phantom

fig=plt.figure();
#B0 map sim phantom
plt.subplot(231); plt.title('True B0 [Hz]', fontsize=15)
plt.imshow(B0*mask_NaN, vmin=-41, vmax=37, origin="lower")
plt.axis('off')
plt.colorbar()

#B1 map sim phantom
plt.subplot(234); plt.title('True B1 [a.u.]', fontsize=15)
plt.imshow(B1*mask_NaN, vmin=0, vmax=1.09, origin="lower")
plt.axis('off')
plt.colorbar()

#B0 map DREAM sim
plt.subplot(232); plt.title('STE DREAM B0 [Hz]', fontsize=15)
plt.imshow(B0_freq*mask_NaN,vmin=-41,vmax=37,origin='lower')
plt.axis('off')
plt.colorbar()

#B1 map DREAM sim
plt.subplot(235); plt.title('STE DREAM B1 [a.u.]', fontsize=15)
plt.imshow(np.abs(B1_angle)*mask_NaN, vmin=0 ,vmax=1.09, origin='lower')
plt.axis('off')
plt.colorbar()

# B0 diff map
plt.subplot(233); plt.title('STE B0 - True B0 [Hz]', fontsize=15)
plt.imshow(-(B0-(B0_freq*mask_NaN)), vmin=-5, vmax=5, origin="lower")
plt.axis('off')
plt.colorbar()
'''
#B1 diff map
plt.subplot(247); plt.title('B1 diff map (DREAM-phantom)', fontsize=15)
plt.imshow(-(B1-(np.abs(B1_angle)*mask_NaN)), vmin=-0.05, vmax=0.05, origin="lower")
plt.colorbar()
'''
'''
# B0 ratio map
plt.subplot(244); plt.title('B0 ratio map (DREAM/phantom)', fontsize=15)
plt.imshow((B0_freq*mask_NaN)/B0, vmin=-2, vmax=2, origin="lower")
plt.colorbar()
'''
#B1 ratio map
plt.subplot(236); plt.title('STE B1 / True B1', fontsize=15)
plt.imshow((np.abs(B1_angle)*mask_NaN)/B1, vmin=0.95, vmax=1.05, origin="lower")
plt.axis('off')
plt.colorbar()


#B0 scatter plot
fig=plt.figure(figsize=(10,6));
plt.subplot(111); #plt.title('B0 comparison', fontsize=15)
n = np.linspace(-55, 55, 111)
slope1, intercept1, _, _, _ = linregress(np.reshape(B0*mask_zero,4096),np.reshape((B0_freq*mask_zero),4096))
plt.xlabel('True B0 [Hz]', fontsize=15)
plt.ylabel('STE DREAM B0 [Hz]', fontsize=15)
plt.xlim(-55,55)
plt.ylim(-55,55)
plt.grid()
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.plot(np.reshape(B0,4096),np.reshape((B0_freq*mask_NaN),4096),'x', label='data')
plt.plot(n, n, color='0.7', linestyle='--', label='optimal match')
plt.plot(n, slope1*n+intercept1, 'r', label='linear fit')
plt.legend(fontsize=15)

#B1 scatter plot
fig=plt.figure(figsize=(10,6));
plt.subplot(111); #plt.title('B1 comparison', fontsize=15)
n = np.linspace(0, 1.2, 121)
slope2, intercept2, _, _, _ = linregress(np.reshape(B1*mask_zero,4096),np.reshape((np.abs(B1_angle)*mask_zero),4096))
plt.ylabel('STE DREAM B1 [a.u.]', fontsize=15)
plt.xlabel('True B1 [a.u.]', fontsize=15)
plt.xlim(0.6,1.2)
plt.ylim(0.6,1.2)
plt.grid()
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.plot(np.reshape(B1,4096),np.reshape((np.abs(B1_angle)*mask_NaN),4096),'x', label='data')
plt.plot(n, n, color='0.7', linestyle='--', label='optimal match')
plt.plot(n, slope2*n+intercept2, 'r', label='linear fit')
plt.legend(fontsize=15)
